In [1]:
import geopandas as gpd
import pydeck as pdk
import numpy as np

In [29]:
gdf = gpd.read_file("Porto_Alegre__Brasil_resolution_8.geojson").to_crs("EPSG:4326")
centroid = gdf.unary_union.centroid
gdf["dis"] = gdf.centroid.distance(centroid)
gdf["normalized_dis"] = (gdf["dis"] - gdf["dis"].min()) / (
    gdf["dis"].max() - gdf["dis"].min()
)
gdf.head()

/tmp/ipykernel_117921/3554204681.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["dis"] = gdf.centroid.distance(centroid)
/tmp/ipykernel_117921/3554204681.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["dis"] = gdf.centroid.distance(centroid)


,h,geometry,dis,normalized_dis
0,88a90e9267fffff,"POLYGON ((-51.18813 -29.99024, -51.18397 -29.9...",0.113105,0.575227
1,88a9012b33fffff,"POLYGON ((-51.15875 -30.09124, -51.15459 -30.0...",0.010550,0.038848
2,88a901283bfffff,"POLYGON ((-51.20294 -30.05559, -51.19879 -30.0...",0.059845,0.296669
3,88a9012d2dfffff,"POLYGON ((-51.24886 -29.98783, -51.24471 -29.9...",0.141436,0.723405
4,88a9012953fffff,"POLYGON ((-51.15796 -30.07190, -51.15381 -30.0...",0.029277,0.136793


In [40]:
colors = [
    [26, 150, 65, 140],
    [166, 217, 106, 140],
    [253, 174, 97, 140],
    [215, 25, 28, 140],
]


def map_value_to_color(value, colors):
    index = int(value * (len(colors) - 1))
    return colors[index]


def map_value_to_elevation(value, min_elevation=0, max_elevation=1000):
    return value * max_elevation

def add_number_of_firefighters(color):
    if color == colors[0]:
        return np.random.randint(0, 1)
    elif color == colors[1]:
        return 1 + np.random.randint(0, 2)
    elif color == colors[2]:
        return 2 + np.random.randint(0, 3)
    else:
        return 5 + np.random.randint(0, 7)


def add_number_of_desaparecidos():
    return np.random.randint(0, 100)

def add_number_of_resgatados(row):
    color = row["color"]
    if color == colors[0]:
        return row["desaparecidos"]
    elif color == colors[1]:
        return max(row["desaparecidos"] - np.random.randint(0, 5), 0)
    elif color == colors[2]:
        return max(row["desaparecidos"] - np.random.randint(0, 25), 0)
    else:
        return max(row["desaparecidos"] - np.random.randint(0, 50), 0)


gdf["color"] = gdf["normalized_dis"].apply(lambda x: map_value_to_color(x, colors))
gdf["elevation"] = gdf["dis"].apply(lambda x: map_value_to_elevation(x))
gdf["firefighters"] = gdf["color"].apply(lambda x: add_number_of_firefighters(x))
gdf["desaparecidos"] = gdf.apply(lambda x: add_number_of_desaparecidos(), axis=1)
gdf["resgatados"] = gdf.apply(lambda x:  add_number_of_resgatados(x), axis=1)

geojson_data = gdf.__geo_interface__

layer = pdk.Layer(
    "GeoJsonLayer",
    geojson_data,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    wireframe=True,
    get_fill_color="properties.color",
    get_line_color=[0, 0, 0],
    get_line_width=50,
    # get_elevation="properties.elevation",
)

view_state = pdk.ViewState(
    longitude=gdf.geometry.centroid.x.mean(),
    latitude=gdf.geometry.centroid.y.mean(),
    zoom=10,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36,
)

r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={
        "text": "{firefighters} bombeiros na área.\n{desaparecidos} pessoas desaparecidas na área.\n{resgatados} pessoas resgatadas."
    },
)
r.to_html("../webapp/resgate.html")

/tmp/ipykernel_117921/1665395787.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude=gdf.geometry.centroid.x.mean(),
/tmp/ipykernel_117921/1665395787.py:67: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude=gdf.geometry.centroid.y.mean(),


In [1]:
html_content = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body, html {
            margin: 0;
            padding: 0;
            width: 100%;
            height: 100%;
            overflow: hidden;
        }
        #map-container {
            position: relative;
            width: 100%;
            height: 100%;
        }
        #overlay {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.2); /* Optional: Add a semi-transparent black background */
        }
        #videoOverlay {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            opacity: 0.25;
            pointer-events: none; /* Ensures the video overlay does not interfere with map interactions */
        }
    </style>
</head>
<body>
  <div id="map-container">
    <iframe src="resgate.html" style="width:100%; height:100%; border:none;"></iframe>
    <video id="videoOverlay" autoplay loop muted>
      <source src="../videos/chuva_2.mp4" type="video/mp4">
      Your browser does not support the video tag.
    </video>
  </div>
</body>
</html>
"""

# Save the HTML content to a file
with open("../webapp/resgate_rain.html", "w") as file:
    file.write(html_content)